# Brain Image Library
You have been given access to a database where you can pull information from one table. The purpose of this exercise is to

* clean the local copy of the table
* compute some basic information about the files in the table
* add these features to the table
* use the original raw data to create a second table
* use these 2 tables to make some pretty plots

In theory, the file and dataset level features you will be computing in this exercise will be stored in a database and used for a dashboard.

## Exercise 1
a. The variable `file` belows points to a CSV file with the file informations. Load the table into the workspace using Pandas or Dask.

**Hint**
If you use Pandas, set `low_memory` to `False`. For more info click [here](https://stackoverflow.com/questions/58551446/how-to-set-low-memory-to-false).

In [ ]:
# INSERT CODE HERE
import pandas as pd
file = '/bil/workshops/2022/data-science/final_project_dataset.csv'
file = 'final_project_dataset.csv'
df = pd.read_csv( file, sep=',', low_memory=False)

b. Print the column names.

In [ ]:
# INSERT CODE HERE
df.keys()

c. This dataframe has one line per file in the file system. The data wrangler already included some file level statistics in the table. However before we can proceed adding more columns, we need to fix some things.

For example, if you take a look at the file creation dates, these look like

```
df['file_creation_date'][0]
'2019-12-0503:26:25'
```

when in fact these should look like 

```
'2019-12-05 03:26:25'
```

First change the values in the column `file_creation_date` so the dates are correct.

Second, change the data type of this series to be a `datetime` object.

**Hint**
* Add a whitespace.

In [ ]:
# INSERT CODE HERE

d. The column `file_size` has the file size in bytes. Add a column named `human_readable_file_size` that is dtype string. This string representation of the file size should be an approximation to the nearest unit with one decimal point. For example, `15M`, `1.7G` and `5.6T`.

In [ ]:
# INSERT CODE HERE

e. The column `xxh128` is empty. To compute this hash and populate this column we will use the function below. However, due to time constraints, we will only compute these hash for files whose extensions are not `.tif`/`.tiff`.

For each file in the table, whose extension is not `.tif`/`.tiff`, compute the hash and store it in the column `xxh128`. Files without this hash, should remain as `None`.

In [ ]:
#INSERT CODE HERE
import subprocess
from pathlib import Path

def compute_xxh128sum( filename ):
    if Path(filename).is_file():
        results = subprocess.check_output('/bil/packages/xxhash/0.8.0/xxh128sum ' + str(filename) + ' | cut -d" " -f1 | xargs', shell=True)
        return results.decode("utf-8").strip()

f. Add column `exists`. Use the value in `filepath` to find if the file exists on disk. If the file exists, then populate the column as `True`. `False` otherwise.

**Hint** 
* Use `Path` from `pathlib`.
* All files should exist, if some of these don't, then report it. Not your fault.

In [ ]:
#INSERT CODE HERE

g. Add columns `download_link`, `download_ready`, `response_code`, `download_timestamp`. Add these four columns and set the default values to `None`.

In [ ]:
#INSERT CODE HERE

h. We can populate the columns above at the same time to minimize the number of requests. In this case, each value in `filepath` can be turned into an download link. For example, the file `/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19032506-191184/readme.txt` can be turned into a URL by replacing the prefix `/bil/data` with `https://download.brainimagelibrary.org`, leading to `https://download.brainimagelibrary.org/00/9c/009c1e6fcc03ebac/mouseID_19032506-191184/readme.txt`.

Download the response header (not the file) to determine if the file is reachable. Add `True` to `download_ready` if the file is reachable. `False` otherwise.

Save the response code from the request to column `response_code` as an integer, e.g. 202, 404, etc.

Record the timestamp when you made this request to column `download_timestamp`. This column should be of dtype `datetime`.

**Hints**
* Write a single method that can perform these tasks in a single call.
* You could also write a method that takes the full dataframe and returns and updated dataframe.
* You should be saving checkpoints to avoid recomputation.
* All links should be reachable. If some of these aren't that's a problem, but not your problem.



In [ ]:
#INSERT CODE HERE

i. Populate the column `dataset_uuid`. This one is tricky. This dataframe has one row per file. These files are grouped together in datasets. However the column `dataset_uuid` is empty. And now we need to populate it.

A dataset can be identified as a combination of `collection_id`, `sample_id` and `directory`. Generate a unique [UUID](https://www.educba.com/python-uuid/) for each unique combination of `collection_id`, `sample_id` and `directory`. Use the UUID to populate the column `dataset_uuid`. Keep in mind there will be multiple rows in this table that will share the combination of `collection_id`, `sample_id` and `directory`, hence all of these rows should also have the `dataset_uuid` as it means these are all part of the same dataset.

In [ ]:
#INSERT CODE HERE

k. If you haven't done it (and you should have saved some checkpoints), then save the df to disk as a pickle file and as a tsv file. Save it to `manifest.tsv` and `manifest.pkl`.

In [ ]:
#INSERT CODE HERE

## Exercise 2
a. Create another dataframe with columns `['dataset_uuid', 'collection_id', 'dataset_id', 'sample_id', 'directory', 'file_extensions','timestamp']`.

i. Set `file_extensions` to `None`.

ii. Use the values in the first dataframe to populate the other columns.

iii. Record the timestamp when you made this request to column `timestamp`. This column should be of dtype `datetime`.

**Hints**
* The first dataframe had a row per each file in the dataset. Whereas the second dataframe should have one row per dataset.

In [ ]:
#INSERT CODE HERE

b. For each dataset in the first dataframe, count the number of file extensions.

i. Store these values as JSON in the column `file_extensions`. For example, if a dataset has 100 `.tif`s and 1000 `.jpeg`s, then you should store the string `{'tif':100,'jpeg':1000 }`.

ii. If a dataset has a file extension that is `None` or `Nan` store it as `other`. For example, `{'tif':100,'jpeg':1000, 'other':4 }`.

In [ ]:
#INSERT CODE HERE

c. Save the dataframe to disk as a pickle file and as a tsv file. Save it to `datasets.tsv` and `datasets.pkl`.

In [ ]:
#INSERT CODE HERE

## Exercise 3
a. Create another dataframe with columns `['timestamp', 'number_of_datasets', 'number_of_collections', 'number_of_files', 'file_extensions']`.

i. Timestamp records the current date, not the time, e.g. 2022-01-23.

ii. The number of datasets in the `datasets` dataframe goes in `number_of_datasets`.

iii. The number of unique collections in the `datasets` dataframe goes in `number_of_collections`.

iv. The total number of files in the `manifest` dataframe goes in `number_of_files`.

v. Another tricky one. Aggregate all the file extensions in column `file_extensions` in dataframe `datasets`, collect these as a single JSON block and it to `file_extensions` in this dataframe.

In [ ]:
#INSERT CODE HERE

b. Save the dataframe to disk as a pickle file and as a tsv file. Save it to `info.tsv` and `info.pkl`.

In [ ]:
#INSERT CODE HERE

## Exercise 4
Create some pretty plots and embed them in this notebook. Feel free to use any plotting library in Python. Make them pretty though. 

a. Waffleplot. Create a waffleplot from the column `file_extensions` in the dataframe `info`. 

* Title should be the date in `timestamp`.
* No axis labels.
* Add legend.

**Hints***
* Use `pywaffle`.

In [ ]:
#INSERT CODE HERE

b. Histogram. Make a histogram using the column `download_ready` from the dataframe `manifest`. 

* Set title to `Broken links`
* Values set to `False` should be labeled as `Broken`.
* Values set to `True` should be labeled as `Not Broken`.
* No legend.
* Set y-label to `Number of links`

In [ ]:
#INSERT CODE HERE

c. Pie chart. Create a pie chart using the values in `status_code` in the dataframe `manifest`. 

* Title should be `Status codes`.
* Add legends.

In [ ]:
#INSERT CODE HERE

d. Create a plot using the values in `file_creation_date` in dataframe manifest. However, this exercise is open-ended. 

This is my user-story and your job is to create the best plot you think will show what I want.

* The only part I care about the `file_create_date` is the date. Or months, or years, not sure.
* What I want to do is to create a plot that I can show the increment of data through the years.
* I am still debating whether I should just show how many files are available per year, or do a cumulative plot since it should be, in theory, monotonically increasing.

What do you think? Make a plot, convince me your plot is the best.

In [ ]:
#INSERT CODE HERE